# LCTSC 2017 DICOM Interface example

To demenostrate the use of the DICOM Interface we will use the 'Lung CT Segmentation Challenge (LCTSC)' dataset, which 
was provided as part of the AAPM challenge in 2017.
The dataset consist of 60 cases, including computed tomography (CT) images and Radiation Therapy (RT) structure sets in DICOM format. The following structures have been delineated:

- Esophagus
- Heart
- Left Lung
- Right Lung
- Spinal Cord

If you would like to use the dataset for your own experiments, it can be downloaded here:

https://wiki.cancerimagingarchive.net/display/Public/Lung+CT+Segmentation+Challenge+2017

In our experiment, we will use 2 structures, and demonstrate how the DICOM interface can be used to load CT scans as well as segmentation masks from the DICOM images. This can be done 'on the fly' or by first reading all batches and saving them to disk.

## Set up tensorflow and GPUs

We start by setting up tensorflow and the GPU we want to use. Here, I decided to run everything on a single GPU.

In [9]:
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1, 2"

## DICOM interface

Now, we import our newly created DICOM interface and initialize it with the path to the LCTSC dataset.

The only reason this is done, is to extract information about the structures and their names contained in the dataset. 
If you already now the names of the ROI's you are interested in, you can jump to the next step.

In [10]:
# Library import
from miscnn.data_loading.interfaces.dicom_io import DICOM_interface

interface = DICOM_interface()

data_path = "LCTSC/"

samples = interface.initialize(data_path)

structures = interface.get_ROI_names(samples[0])

print('Found structures in sample : {}'.format(structures))


Found structures in sample : [["1", 'Esophagus'], ["2", 'Heart'], ["3", 'Lung_L'], ["4", 'Lung_R'], ["5", 'SpinalCord']]


We can see, that we found 5 different structures for the first sample (patient) in our dataset. 
In this example we assume that we want to build a model that performs semantic segmentation of the
left and right lung. 
In order to tell the DICOM interface, which structures we are interested of, we simpy create a dictionary, containing the
ROI's, as well as their class labels. The class labels are then used as the segmentation pixel values.

Here we assign class: 1 to the left lung and class: 2 to the right lung. Notice that class: 0 is always used as the background class, so the dictionary should start with class 1.

In [11]:
structure_dict = {"Lung_L": 1, "Lung_R": 2}

print(structure_dict)

{'Lung_L': 1, 'Lung_R': 2}


Now that we know the names of the ROI's and have decided which structures we want to use in our model, we simply create
a new instance of the DICOM interface, and provide it with the structure dictionary and the number of classes.

In [12]:
interface = DICOM_interface(structure_dict = structure_dict, classes=3)

## Connection the DICOM interface to the DATA_IO class

Now that we have created our DICOM interface, we simply connect it to the Data_IO class.

In [14]:
from miscnn.data_loading.data_io import Data_IO

data_io = Data_IO(interface, data_path)

We can check the interface by getting a list of all samples.

In [15]:
sample_list = data_io.get_indiceslist()
sample_list.sort()
print("All samples: " + str(sample_list))

All samples: ['LCTSC-Test-S1-101', 'LCTSC-Test-S1-102', 'LCTSC-Test-S1-103', 'LCTSC-Test-S1-104', 'LCTSC-Test-S1-201', 'LCTSC-Test-S1-202', 'LCTSC-Test-S1-203', 'LCTSC-Test-S1-204', 'LCTSC-Test-S2-101', 'LCTSC-Test-S2-102', 'LCTSC-Test-S2-103', 'LCTSC-Test-S2-104', 'LCTSC-Test-S2-201', 'LCTSC-Test-S2-202', 'LCTSC-Test-S2-203', 'LCTSC-Test-S2-204', 'LCTSC-Test-S3-101', 'LCTSC-Test-S3-102', 'LCTSC-Test-S3-103', 'LCTSC-Test-S3-104', 'LCTSC-Test-S3-201', 'LCTSC-Test-S3-202', 'LCTSC-Test-S3-203', 'LCTSC-Test-S3-204', 'LCTSC-Train-S1-001', 'LCTSC-Train-S1-002', 'LCTSC-Train-S1-003', 'LCTSC-Train-S1-004', 'LCTSC-Train-S1-005', 'LCTSC-Train-S1-006', 'LCTSC-Train-S1-007', 'LCTSC-Train-S1-008', 'LCTSC-Train-S1-009', 'LCTSC-Train-S1-010', 'LCTSC-Train-S1-011', 'LCTSC-Train-S1-012', 'LCTSC-Train-S2-001', 'LCTSC-Train-S2-002', 'LCTSC-Train-S2-003', 'LCTSC-Train-S2-004', 'LCTSC-Train-S2-005', 'LCTSC-Train-S2-006', 'LCTSC-Train-S2-007', 'LCTSC-Train-S2-008', 'LCTSC-Train-S2-009', 'LCTSC-Train-S2-010'

## Data augmentation

Like described in the Kidney segmentation example, we can set up a data augmentation pipeline.

In [16]:
# Library import
from miscnn.processing.data_augmentation import Data_Augmentation

# Create and configure the Data Augmentation class
data_aug = Data_Augmentation(cycles=2, scaling=True, rotations=True, elastic_deform=True, mirror=True,
                             brightness=True, contrast=True, gamma=True, gaussian_noise=True)

## Choose preprocessing functions

Now we set up some preprocssing functions. We first normalize the images, resample the scans to same voxel size and clip values that fall out of a certain range.

In [17]:
from miscnn.processing.subfunctions.normalization import Normalization
from miscnn.processing.subfunctions.clipping import Clipping
from miscnn.processing.subfunctions.resampling import Resampling

# Create a pixel value normalization Subfunction through Z-Score 
sf_normalize = Normalization()

sf_resample = Resampling((3.22, 1.62, 1.62))
# Create a clipping Subfunction between -79 and 304
sf_clipping = Clipping(min=-79, max=304)
# Create a resampling Subfunction to voxel spacing 3.22 x 1.62 x 1.62
# Assemble Subfunction classes into a list
# Be aware that the Subfunctions will be exectued according to the list order!
subfunctions = [sf_resample, sf_clipping, sf_normalize]

We then create our Preprocessor and decide how we would like to train our network (full image or patch wise). In this case we use the patchwise-crop analysis, where we randomly crop patches of size (80, 160, 160) from our CT volumes. 

In [18]:
# Library import
from miscnn.processing.preprocessor import Preprocessor

# Create and configure the Preprocessor class
pp = Preprocessor(data_io, data_aug=data_aug, batch_size=2, subfunctions=subfunctions, prepare_subfunctions=False, 
                  prepare_batches=True, analysis="patchwise-crop", patch_shape=(80, 160, 160))

## Building the Neural Network

Finally we decide which neural network archictecture we want to use, which loss function we want to optimize and which metrics we want to use as a performance measure.

In [19]:
# Library import
from miscnn.neural_network.model import Neural_Network
from miscnn.neural_network.metrics import dice_soft, dice_crossentropy, tversky_loss

# Create the Neural Network model
model = Neural_Network(preprocessor=pp, loss=tversky_loss, metrics=[dice_soft, dice_crossentropy],
                       batch_queue_size=3, workers=3, learninig_rate=0.0001, gpu_number=1)



Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [20]:
from miscnn.evaluation import split_validation

split_validation(sample_list, model,epochs = 200)

KeyboardInterrupt: 